In [27]:
import pandas as pd
import os

train_dir_path = "deft_corpus/data/deft_files/train_converted_task1"
train_deft_files = os.listdir(train_dir_path)
train_dataframe = pd.DataFrame([])
for file in train_deft_files:
    dataframe = pd.read_csv(os.path.join(train_dir_path, file), sep="\t", header = None)
    dataframe.columns = ["Sentence","HasDef"]
    train_dataframe = train_dataframe.append(dataframe, ignore_index=True)

In [12]:
train_dataframe["HasDef"].value_counts()

0    12143
1     6014
Name: HasDef, dtype: int64

In [13]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [14]:
train_dataframe["parsed"] = train_dataframe.Sentence.apply(nlp)

26

In [145]:
test = nlp(train_dataframe["Sentence"][1])
#print ([token.text for token in test])
#print(list(test.sents))
' '.join([token.lemma_ for token in test if not token.is_stop and not token.is_punct and not token.is_space and token.is_alpha])

'memorize fact rehearsal strategy chunk organize information manageable bit chunk Bodie Powers Fitch Hauser'

In [153]:
processed_dataframe = train_dataframe.copy()
for index in range(len(train_dataframe)):
    about_sentence = nlp(train_dataframe["Sentence"][index])
    processed_dataframe.iloc[index,0] = [token.lemma_ for token in about_sentence 
                                                   if not token.is_stop and not token.is_punct and 
                                                   not token.is_space and token.is_alpha]

In [167]:
processed_dataframe["Sentence"][10006]

'Technotron new technology case import good county firm industry lose money lie worker'

In [166]:
train_dataframe["Sentence"][10006]

' 5246 . Because of Technotron ’s new technology — which in this case is importing goods from another county — other firms in this industry will lose money and lay off workers .'

In [175]:
train_X = []
for sentence in nlp.pipe(train_dataframe['Sentence'].astype('unicode').values):
    train_X.append(sentence.tensor)

In [176]:
import numpy as np
train_X = np.array(train_X)

In [192]:
train_X.shape

(18157,)

In [194]:
def is_token_allowed(token):
    '''
        Only allow valid tokens which are not stop words
        and punctuation symbols.
    '''
    if (not token or not token.string.strip() or
        token.is_stop or token.is_punct):
        return False
    return True

def preprocess_token(token):
    # Reduce token to its lowercase lemma form
    return token.lemma_.strip().lower()

complete_doc = 
complete_filtered_tokens = [preprocess_token(token)
    for token in complete_doc if is_token_allowed(token)]
complete_filtered_tokens

In [196]:
train_dataframe["processed_tokens"][0]

'4309'

In [37]:
corpus_path = "deft_corpus/data"
train_output_path = os.path.join(corpus_path, "deft_files/converted_train")

In [38]:
if not (os.path.exists(train_output_path)):
    os.mkdir(train_output_path)

In [1]:
from source.data_loader import DeftCorpusLoader

In [2]:
loader = DeftCorpusLoader("deft_corpus/data")

In [4]:
loader.convert_to_classification_format()

In [5]:
train_dataframe, dev_dataframe = loader.load_classification_data("deft_corpus/data/deft_files/converted_train", "deft_corpus/data/deft_files/converted_dev")

In [30]:
import spacy
nlp = spacy.load('en_core_web_sm')
from spacy.lang.en import English

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our tokens object, which is used to create documents with linguistic annotations.
    tokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    # Removing stop words, punctations, spaces and non alphanumeric characters.
    tokens = [ token.lemma_.lower().strip() for token in tokens 
                if not token.is_stop and not token.is_punct and 
                    not token.is_space and token.is_alpha ]
    
    # return preprocessed list of tokens
    return tokens

In [31]:
train_dataframe["Parsed"] = train_dataframe.Sentence.apply(spacy_tokenizer)

In [32]:
train_dataframe

,Sentence,HasDef,parsed
0,"3918 . You may recall that 6 x 6 = 36 , 6 x 7...",0,"[recall, x, x, x]"
1,Memorizing these facts is rehearsal . Another...,1,"[memorizing, fact, rehearsal, strategy, chunk,..."
2,Chunking is useful when trying to remember in...,0,"[chunking, useful, try, remember, information,..."
3,3921 . Use elaborative rehearsal : In a famou...,1,"[use, elaborative, rehearsal, famous, article,..."
4,Their theory is called levels of processing .,0,"[theory, call, level, process]"
...,...,...,...
18152,The term culture refers to all of the beliefs...,1,"[term, culture, refer, belief, custom, art, tr..."
18153,Culture is transmitted to people through lang...,0,"[culture, transmit, people, language, model, c..."
18154,4306 . Another method for assessment of perso...,0,"[method, assessment, personality, projective, ..."
18155,This kind of test relies on one of the defens...,1,"[kind, test, rely, defense, mechanism, propose..."
